In [ ]:
import os
import requests

html_filename = "Guten-Rutsch-und-ein-gesundes-neues-Jahr-2021-5001311.html"
if not os.path.isfile(html_filename):
    print("download")
    r = requests.get(f"https://www.heise.de/news/{html_filename}")
    open(html_filename, 'wb').write(r.content)

In [ ]:
from bs4 import BeautifulSoup
html = open(html_filename).read()
soup = BeautifulSoup(html)

In [ ]:
d = {}

In [ ]:
d["title"] = soup.h1.text.strip()
d["header"] = soup.select_one("#meldung > div.article-layout__header-container > header > p").text.strip()
d["author"] = soup.select_one("a.redakteurskuerzel__link").attrs["title"]

In [ ]:
d["text"] = "\n".join([p.text.strip() 
                            for p in soup.select("#meldung > div.article-layout__content-container > div > p")])

In [ ]:
import json
ld = json.loads(soup.find("script", type="application/ld+json").string)
for k in ["identifier", "url", "datePublished", "commentCount"]:
    d[k] = ld[k]

In [ ]:
d

In [ ]:
import pandas as pd
articles = pd.DataFrame([d]).set_index("identifier")
articles

In [ ]:
articles.T

In [ ]:
articles.to_csv()

In [ ]:
articles.to_json(orient='records')

In [ ]:
import sqlite3
sql = sqlite3.connect("heise-articles.db")
articles.to_sql("articles", sql, index_label="id", if_exists="replace")